In [1]:
from pyspark import SparkContext

In [2]:
import os

In [3]:
sc = SparkContext("local","app")

21/12/02 02:52:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/02 02:52:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [24]:
dir = '/temp/Data'
files = []

In [25]:
# Fetches files from all directories and sub directories recursively
def fetchFileRec(dir, files):
    for file in os.listdir(dir):
        floc = os.path.join(dir,file)
        print(floc)
        if(os.path.isfile(floc)):
            files.append(floc)
        else:
            fetchFileRec(floc,files)
            
    return files

In [26]:
fetchFileRec(dir,files)

/temp/Data/README
/temp/Data/glossary
/temp/Data/tragedies
/temp/Data/tragedies/titusandronicus
/temp/Data/tragedies/othello
/temp/Data/tragedies/antonyandcleopatra
/temp/Data/tragedies/hamlet
/temp/Data/tragedies/timonofathens
/temp/Data/tragedies/macbeth
/temp/Data/tragedies/coriolanus
/temp/Data/tragedies/romeoandjuliet
/temp/Data/tragedies/juliuscaesar
/temp/Data/tragedies/kinglear
/temp/Data/Tolstoy
/temp/Data/Tolstoy/war_and_peace.txt
/temp/Data/Tolstoy/anna_karenhina.txt
/temp/Data/poetry
/temp/Data/poetry/venusandadonis
/temp/Data/poetry/sonnets
/temp/Data/poetry/rapeoflucrece
/temp/Data/poetry/loverscomplaint
/temp/Data/poetry/various
/temp/Data/Hugo
/temp/Data/Hugo/NotreDame_De_Paris.txt
/temp/Data/Hugo/Miserables.txt
/temp/Data/comedies
/temp/Data/comedies/tamingoftheshrew
/temp/Data/comedies/midsummersnightsdream
/temp/Data/comedies/muchadoaboutnothing
/temp/Data/comedies/periclesprinceoftyre
/temp/Data/comedies/twelfthnight
/temp/Data/comedies/measureforemeasure
/temp/Data

['/temp/Data/README',
 '/temp/Data/glossary',
 '/temp/Data/tragedies/titusandronicus',
 '/temp/Data/tragedies/othello',
 '/temp/Data/tragedies/antonyandcleopatra',
 '/temp/Data/tragedies/hamlet',
 '/temp/Data/tragedies/timonofathens',
 '/temp/Data/tragedies/macbeth',
 '/temp/Data/tragedies/coriolanus',
 '/temp/Data/tragedies/romeoandjuliet',
 '/temp/Data/tragedies/juliuscaesar',
 '/temp/Data/tragedies/kinglear',
 '/temp/Data/Tolstoy/war_and_peace.txt',
 '/temp/Data/Tolstoy/anna_karenhina.txt',
 '/temp/Data/poetry/venusandadonis',
 '/temp/Data/poetry/sonnets',
 '/temp/Data/poetry/rapeoflucrece',
 '/temp/Data/poetry/loverscomplaint',
 '/temp/Data/poetry/various',
 '/temp/Data/Hugo/NotreDame_De_Paris.txt',
 '/temp/Data/Hugo/Miserables.txt',
 '/temp/Data/comedies/tamingoftheshrew',
 '/temp/Data/comedies/midsummersnightsdream',
 '/temp/Data/comedies/muchadoaboutnothing',
 '/temp/Data/comedies/periclesprinceoftyre',
 '/temp/Data/comedies/twelfthnight',
 '/temp/Data/comedies/measureforemeasur

In [35]:
rdd = None
stopWords = ['they','she','he','it','the','as','is','and']

for floc in files:
    file = sc.wholeTextFiles(floc)
    f_rdd = file.flatMap(lambda p: [(p[0], word) for word in p[1].lower().split()]) \
            .filter(lambda p: p[1] not in stopWords) \
            .map(lambda p: ((p[1],p[0]), 1)).reduceByKey(lambda a,b: a+b)
    
    if rdd:
        rdd = rdd.union(f_rdd)
    else:
        rdd = f_rdd
    

In [36]:
output_rdd = rdd.map(lambda x: (x[0][0], (x[0][1], x[1]))).groupByKey().mapValues(list)
output_rdd.take(10)

[('this',
  [('file:/temp/Data/README', 1),
   ('file:/temp/Data/glossary', 2),
   ('file:/temp/Data/tragedies/titusandronicus', 192),
   ('file:/temp/Data/tragedies/othello', 183),
   ('file:/temp/Data/tragedies/antonyandcleopatra', 139),
   ('file:/temp/Data/tragedies/hamlet', 249),
   ('file:/temp/Data/tragedies/timonofathens', 105),
   ('file:/temp/Data/tragedies/macbeth', 94),
   ('file:/temp/Data/tragedies/coriolanus', 145),
   ('file:/temp/Data/tragedies/romeoandjuliet', 190),
   ('file:/temp/Data/tragedies/juliuscaesar', 141),
   ('file:/temp/Data/tragedies/kinglear', 206),
   ('file:/temp/Data/Tolstoy/war_and_peace.txt', 1835),
   ('file:/temp/Data/Tolstoy/anna_karenhina.txt', 1170),
   ('file:/temp/Data/poetry/venusandadonis', 40),
   ('file:/temp/Data/poetry/sonnets', 91),
   ('file:/temp/Data/poetry/rapeoflucrece', 107),
   ('file:/temp/Data/poetry/loverscomplaint', 7),
   ('file:/temp/Data/poetry/various', 11),
   ('file:/temp/Data/Hugo/NotreDame_De_Paris.txt', 933),
   ('